In [ ]:
# Step 2: Set up Kaggle API credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Step 3: Install the Kaggle library (if not already installed)
!pip install kaggle

# Step 4: Download the Customer Life Time Value dataset
!kaggle datasets download shibumohapatra/customer-life-time-value

# Step 5: Extract the downloaded zip file
import zipfile
with zipfile.ZipFile("customer-life-time-value.zip", 'r') as zip_ref:
    zip_ref.extractall("customer-life-time-value")

# Step 6: List the extracted files to confirm 'train_BRCpofr.csv' is present
!ls customer-life-time-value

# Step 7: Load the 'train_BRCpofr.csv' file into a Pandas DataFrame
import pandas as pd
ds = pd.read_csv("customer-life-time-value/train_BRCpofr.csv")


In [ ]:
ds.shape

In [ ]:
(1.6-1.8)/1.6

In [ ]:
ds.sample(10)

In [ ]:
ds.info()

In [ ]:
ds.describe()

In [ ]:
ds.columns

In [ ]:
ds['gender'].value_counts().plot(kind='bar')

In [ ]:
ds['area'].value_counts().plot(kind='bar')

In [ ]:
ds['qualification'].value_counts()

In [ ]:
ds['income'].value_counts()

In [ ]:
ds['marital_status'].value_counts()

In [ ]:
ds['vintage'].value_counts().plot(kind='bar')

In [ ]:
ds['claim_amount'].plot(kind='kde')

In [ ]:
ds[ds['claim_amount']==0].shape

In [ ]:
ds[ds['claim_amount']!=0]['claim_amount'].mean()

In [ ]:
ds[ds['claim_amount']!=0]['claim_amount'].median()

In [ ]:
ds['num_policies'].value_counts()

In [ ]:
ds['policy'].value_counts()

In [ ]:
ds['type_of_policy'].value_counts().plot(kind='pie',autopct='%1.1f%%')

In [ ]:
ds['cltv'].describe()

In [ ]:
ds['cltv'].plot(kind='kde')

In [ ]:
ds.columns

In [ ]:
# income column range inta single number
# check for the relationships between the other column and estimate the close limit.
#1. num_policies have the direct correlation between the income
#2. 5-10L 60% male with respect to all the males where as 57% females  as well as 26% females where as 16% males for income less than 2 lpa

In [ ]:
ds[ds['income']=='More than 10L'][['num_policies']].value_counts()

In [ ]:
ds[['income','num_policies']].sample(30)

In [ ]:
gender = ds.groupby('gender')

In [ ]:
gender['income'].value_counts()

In [ ]:
(ds[ds['gender']=='Female']).shape

In [ ]:
22220 + 9493 + 6144 + 1038

In [ ]:
(gender['income'].value_counts()['Female']/38895)

In [ ]:
(gender['income'].value_counts()['Male'])/50497

In [ ]:
(gender['income'].value_counts()['Female']/38865).plot(color='Black',label='Female')
(gender['income'].value_counts()['Male']/50497).plot(color='Red',label='Male')


In [ ]:
# Qualificaton
# not conclusive
quali= ds.groupby('qualification')
quali['income'].value_counts()

In [ ]:
quali= ds.groupby('marital_status')
quali['income'].value_counts()

In [ ]:
# marital_status
def income_per(ds,var):
  temp_df=pd.DataFrame(columns=['status','income','distribution_percent'])
  for i in ds[var].unique():
    T=pd.DataFrame(columns=['status','income','distribution_percent'])
    t=ds[ds[var]==i]
    for j in t['income'].unique():
      Tt=(t[t['income']==j].shape[0]/t.shape[0])
      T[['status','income','distribution_percent']] = [[j,i,Tt]]
      temp_df =pd.concat([temp_df,T],ignore_index=True)

  return temp_df.set_index('status').pivot(columns='income')
income_per(ds,'marital_status')
#income_percentage.set_index('status').pivot(columns='income')

In [ ]:
# Area
income_per(ds,'area')

In [ ]:
area=ds.groupby('area')
t = area['income'].value_counts()
t

In [ ]:
t['Rural']/t['Rural'].values.sum()

In [ ]:
t['Urban']/t['Urban'].values.sum()

In [ ]:
ds.columns


In [ ]:
income_per(ds,'policy')

In [ ]:
# type_of_policy
# quite descent realationship
income_per(ds,'type_of_policy')

In [ ]:
inc = ds.groupby('income')
inc['cltv'].mean()

In [ ]:
ds.columns

In [ ]:
ds['claim_amount'].describe()

In [ ]:
sigma = ds['claim_amount'].std()
mean = ds['claim_amount'].mean()
upperlimit = mean  + 3 * (sigma)
lowerlimit = mean - 3*(sigma)
print(upperlimit,lowerlimit)

In [ ]:
sigma/mean

In [ ]:
ds = ds[((ds['claim_amount']>lowerlimit) & (ds['claim_amount'] < upperlimit))]

In [ ]:
# basic training
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler,LabelEncoder
from sklearn.metrics import r2_score,mean_absolute_error,mean_absolute_error

In [ ]:
X=ds.iloc[:,1:-1]
y=ds.iloc[:,-1]

In [ ]:
X_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.8)

In [ ]:
ds['type_of_policy'].unique()

In [ ]:
scaling= StandardScaler()

In [ ]:
ohe = OneHotEncoder(drop='first',sparse_output=False)
ord_quali = OrdinalEncoder(categories=[['Others','High School','Bachelor']])
ord_inco = OrdinalEncoder(categories=[['<=2L','2L-5L','5L-10L','More than 10L']])
ord_num_policies = OrdinalEncoder(categories=[['1','More than 1']])
ohe_policy = OneHotEncoder(drop='first')
ord_type_policy = OrdinalEncoder(categories=[['Silver','Platinum','Gold']])

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
ds.head(3)

In [ ]:
tr = ColumnTransformer(
    transformers=[
        #('ohe', ohe, ['gender','area','marital_status']),
        ('ord_quali', ord_quali, ['qualification']),
        ('ord_inco', ord_inco, ['income']),
        ('ord_num_policies', ord_num_policies, ['num_policies']),
        # ('ohe_policy', ohe_policy, ['policy']),
        ('label_gender',label_encoder,['gender']),
        ('label_area',label_encoder,['area']),
        ('label_marital_status',label_encoder,['marital_status']),
        ('label_policy',label_encoder,['policy']),
        ('ord_type_policy', ord_type_policy, ['type_of_policy']),
        ('scaling',scaling,['vintage','claim_amount'])
    ],
    remainder='passthrough'
)

In [ ]:
X_train

In [ ]:
X_train_tr= tr.fit_transform(X_train)
x_test_tr = tr.transform(x_test)


# Feature Importances

In [ ]:
import numpy as np

In [ ]:
X_trans = tr.fit_transform(X)
y_trans = np.log1p(y)

# RandomForest Feature importances

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_trans,y_trans)


In [ ]:
rf.feature_importances_.shape

In [ ]:
X.columns.shape

In [ ]:
rf.feature_importances_
rf_importances = pd.DataFrame({'Features':X.columns,'rf_importances':rf.feature_importances_})
rf_importances

In [ ]:
rf.feature_importances_

In [ ]:
# linear regression
lr = LinearRegression()
lr.fit(X_train_tr,y_train)
y_pred = lr.predict(x_test_tr)
r2_score(y_test,y_pred)

In [ ]:
y_pred_lr_tr= lr.predict(X_train_tr)
r2_score()

In [ ]:
# random forest
rf = RandomForestRegressor()

In [ ]:
rf.fit(X_train_tr,y_train)

In [ ]:
#y_pred = rf.predict(x_test_tr)
y_pred = rf.predict(x_test_tr)

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
# Gradient boosting
Gr = GradientBoostingRegressor()

In [ ]:
Gr.fit(X_train_tr,y_train)

In [ ]:
y_pred_gr = Gr.predict(x_test_tr)

In [ ]:
r2_score(y_test,y_pred_gr)